# Load Data

In [1]:
import music21
import networkx as nx
import matplotlib.pyplot as plt
import numpy as np
import mnet

Create path to access local files

In [2]:
'''
Creat local corpus with access pieces
'''

localCorpus = music21.corpus.corpora.LocalCorpus()
localCorpus.addPath('../library')
music21.corpus.cacheMetadata()

/opt/anaconda3/envs/music-network-env/lib/python3.7/site-packages/music21/corpus/corpora.py: WARNING: local metadata cache: starting processing of paths: 2
/opt/anaconda3/envs/music-network-env/lib/python3.7/site-packages/music21/corpus/corpora.py: WARNING: cache: filename: /var/folders/j6/2q7hftds7vb8_lsfy50l8lzw0000gn/T/music21/local.p.gz
bundles.py: WARNING: MetadataBundle Modification Time: 1593031888.6425247
bundles.py: WARNING: Skipped 2 sources already in cache.
/opt/anaconda3/envs/music-network-env/lib/python3.7/site-packages/music21/corpus/corpora.py: WARNING: cache: writing time: 0.024 md items: 2

/opt/anaconda3/envs/music-network-env/lib/python3.7/site-packages/music21/corpus/corpora.py: WARNING: cache: filename: /var/folders/j6/2q7hftds7vb8_lsfy50l8lzw0000gn/T/music21/local.p.gz
caching.py: WARNING: cache: final writing time: 0.025 seconds


Save Telemann Fantasie No. 1 as a stream object

In [3]:
s = music21.corpus.parse('telemannfantasie1.xml')

Inspect stream to see where desired parts are located

In [4]:
s.show("text")

{0.0} <music21.text.TextBox "IMSLP23680...">
{0.0} <music21.text.TextBox "2">
{0.0} <music21.text.TextBox "3">
{0.0} <music21.text.TextBox "4">
{0.0} <music21.metadata.Metadata object at 0xa1b737e90>
{0.0} <music21.stream.Part Flute>
    {0.0} <music21.instrument.Instrument 'P1: Flute: Flute (2)'>
    {0.0} <music21.stream.Measure 1 offset=0.0>
        {0.0} <music21.layout.PageLayout>
        {0.0} <music21.layout.SystemLayout>
        {0.0} <music21.layout.StaffLayout distance None, staffNumber 1, staffSize None, staffLines None>
        {0.0} <music21.clef.TrebleClef>
        {0.0} <music21.key.Key of A major>
        {0.0} <music21.meter.TimeSignature 4/4>
        {0.0} <music21.note.Note A>
        {1.0} <music21.note.Note A>
        {1.25} <music21.note.Note B>
        {1.5} <music21.note.Note A>
        {1.75} <music21.note.Note E>
        {2.0} <music21.note.Note A>
        {2.25} <music21.note.Note C#>
        {2.5} <music21.note.Note A>
        {2.75} <music21.note.Note E>
  

Save Solo Flute part

In [5]:
flute = s[5]

# Basic Conversion Pipeline

Convert stream to list of notes

In [6]:
flute_notes =flute.recurse().notes

In [7]:
notelst = list(flute_notes)

Convert list of notes to list of nodes for graph

In [8]:
nodelst_basic=mnet.convert_basic(notelst)

Create Multiedge Directional Graph

In [9]:
g_basic=mnet.create_graph(nodelst_basic)

#write as .gexf
#nx.write_gexf(g_basic, "basic_composition.gexf")

In [11]:
from networkx.readwrite import json_graph

data = json_graph.node_link_data( mnet.convert_to_weighted(g_basic))

In [12]:
import json
with open('basic.json', 'w') as outfile:
    json.dump(data, outfile)

Run and Save Random Walk

In [ ]:
def convert_to_stream(notelist):

    s = music21.stream.Stream()
    for thisNote in notelist:
        s.append(thisNote)
    return s


In [ ]:
randomwalk_basic=mnet.generate_randomwalk(g_basic)

In [ ]:
len(randomwalk_basic)

Convert random walk to back to music.

Uses conversion function strto16thnote, which takes a pitch string and converts it to a note with length of 16th note

In [ ]:
tune = mnet.strto16thnote(randomwalk_basic)
new_composition_basic = convert_to_stream(tune)

#Write to MIDI 
file = new_composition_basic.write('xml', "../xml/Basic.xml")


In [ ]:
sp = music21.midi.realtime.StreamPlayer(new_composition_basic)
sp.play()    


# Analysis

### Modularity
    Run in gephi
    G#5 and A5 make sense becasue they have strong bidirectional link and are often paired. 
    Highest in-degree: in-range chord tones
    
    Would expect groups of chord tones
    Page Rank-note composition of final piece
    Eigenvector created more "even field"

In [ ]:
g_basic